In [10]:
# Imports
from wNMF import wNMF
import numpy as np
import matplotlib.pyplot as plt   # allows to plot charts
import pandas as pd              
import seaborn as sns    # python data visualization library based on matplotlib
import glob
import pickle
import os 
# To sort tests per test_id
from collections import OrderedDict

# To zoom in on the plots, uncomment the following lines
# %matplotlib inline
# import mpld3
# mpld3.enable_notebook()

sns.set(style="darkgrid")

### Instantaneous Velocities retrieval

In [11]:
# Return a color map to assign colors to test_ids
def get_cmap(n, name='gist_rainbow'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

# Root directory where the files are stored
path_dir = './logs/simulation_data/Frequencies'

# Loop to recursively look for files with .pkl extension, even in subfolders
pkl_files = {}
current_test = None
for root, dirs, files in os.walk(path_dir):
    if(len(root.split(os.sep)[-2:-1]) >= 1):
        folder_name = root.split(os.sep)[-2:-1][0].split('_')
        if(len(folder_name) == 3 and folder_name[0] == "Test" and folder_name[2] == "Frequencies"):
            if(current_test != folder_name[1]):
                current_test = folder_name[1]
                pkl_files[current_test] = []
            for file in files:
                if "values.pkl" in file:
                    path = "{}{}{}".format(root, os.sep, file)
                    pkl_files[current_test].append(path)
                    
# Sort files per test_id asc
pkl_files = OrderedDict(sorted(pkl_files.items()))

# Color Map, assign a color to a test_id. The first test_id will be represented with the first color of the array aso...
possibleColors = get_cmap(len(pkl_files.keys()) + 1)
test_color = {}

# Open and load the files as dictionaries
# We store those dict in an bigger dict for futur reference, the filename will be the key
current_test = None
data = {}
count = 0
for test_id, files in pkl_files.items():
    if current_test != test_id:
        current_test = test_id
        data[current_test] = {}
        test_color[current_test] = possibleColors(count)
        count += 1
    for pkl in files:
        with open(pkl, 'rb') as f:
            data[current_test][pkl.split(os.sep)[-1].split('.')[-2].split('_')[-1]] = pickle.load(f)
            

#### DATA STRUCTURE #####
# data["516"] -> contains all values files in Test_516_Frequencies
# data["516"]["1.0"] -> contains value file in Test_516_Frequencies with a frequency of 1.0

In [12]:
velocities = {}
# Python2 -> data.iteritems()
for test_nb, _ in data.items():
    velocities[test_nb] = {}
    for freq in sorted(data[test_nb].keys()):
        print(freq, test_nb)
        velocities[test_nb][freq] = [data[test_nb][freq][i]["pelvis"]["vel"][0] for i in range(len(data[test_nb][freq]))]

means = {}
maximums = {}
for test_nb, _ in velocities.items():
    means[test_nb] = {}
    maximums[test_nb] = {}
    for freq, velocity in velocities[test_nb].items():
        means[test_nb][freq] = np.mean(velocity)
        maximums[test_nb][freq] = np.max(velocity)

# Create a table of frequencies
for test_nb, test_val in data.items():
    min = float(sorted(test_val.keys())[0])
    max = float(sorted(test_val.keys())[-1:][0])
    step = len(test_val.keys())

freq = np.array(np.linspace(min, max, step))



values 2
values 3
values 4
values 5
values 6
values 600
values 601


ValueError: could not convert string to float: 'values'

In [ ]:
# Plot frequencies vs Velocities
tmp_mean = []
tmp_freq = []
for test, _ in means.items():
    for f in freq:
        # Floating point classic issue : 1.1 is stored as 1.099999999 https://stackoverflow.com/questions/24670608/why-does-python-show-0-2-0-1-as-0-30000000000000004
        if(len(str(f)) > 4):
            f = round(f, 1)
        if(str(f) in means[test].keys()):
            tmp_freq.append(f)
            tmp_mean.append(means[test][str(f)])
    plt.plot(tmp_freq, tmp_mean, label="Exp. {}".format(test))
    plt.legend()
    tmp_freq = []
    tmp_mean = []

# Changer les noms des fichiers pour avoir le bon numero d'exp
    
    
plt.title('Frequency with respect to velocity')
plt.ylabel('Mean Velocity [m/s]')
plt.xlabel('Frequency [Hz]')
plt.legend(bbox_to_anchor = (1.05, 1), loc=2, prop={'size':14})
plt.show()